In [3]:
import sys
sys.path.append("..")

import geopandas as gpd

from ETL.common import execute_query

In [2]:
q = "SELECT * FROM houses_clean ORDER BY timestamp DESC"
df_raw = execute_query(q)

,fid,Rang,geometry
0,1,40 - 45 dB(A),"MULTIPOLYGON (((424795.052 4576392.601, 424794..."
1,1,40 - 45 dB(A),"MULTIPOLYGON (((424910.719 4576528.069, 424910..."
2,1,40 - 45 dB(A),"MULTIPOLYGON (((424785.062 4582248.117, 424786..."
3,1,40 - 45 dB(A),"MULTIPOLYGON (((425173.809 4583157.486, 425174..."
4,1,40 - 45 dB(A),"MULTIPOLYGON (((425227.94 4583282.291, 425226 ..."
...,...,...,...
111539,82401,>= 80 dB(A),"MULTIPOLYGON (((425308 4576282.987, 425306 457..."
111540,82402,>= 80 dB(A),"MULTIPOLYGON (((425384 4576088.658, 425382 457..."
111541,82403,>= 80 dB(A),"MULTIPOLYGON (((427594.122 4575932.455, 427594..."
111542,82404,>= 80 dB(A),"MULTIPOLYGON (((426838 4575310.481, 426836.253..."


In [4]:
df = df_raw.copy()

In [ ]:
loudness_df = gpd.read_file("2017_isofones_total_lden_mapa_estrategic_soroll_bcn.gpkg",
                     layer="2017_Isofones_Total_Lden_Mapa_Estrategic_Soroll_BCN")
loudness_df = loudness_df.to_crs(epsg=4326)

In [ ]:
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")
df = df.sjoin(loudness_df, how="left")
df = df[df.columns.difference(["fid", "index_right"])]
df = df.rename(columns={"Rang": "loudness"})

In [ ]:
cat_cols = ["active", "city", "type", "orientation", "age", "parking", "floor", 
            "elevator", "furniture", "state", "heating", "water_heating", 
            "formatted_address", "pets", "loudness"]
num_cols = ["price", "rooms", "bathrooms", "surface", "latitude", "longitude"]

plot_cols = cat_cols + num_cols

for c in cat_cols:
    df[c] = df[c].fillna("Null")
    if c == "age":
        df[c] = pd.Categorical(df[c], ["Null", "Menos de 1 año", "1 a 5 años", "5 a 10 años", "10 a 20 años", 
                                       "20 a 30 años", "30 a 50 años", "70 a 100 años", "+ 100 años"])
    elif c == "floor":
        df[c] = pd.Categorical(df[c], ["Null", "Sótano", "Otro", "Bajos", "Entresuelo", "1ª planta", "2ª planta",
                                       "3ª planta", "4ª planta", "5ª planta", "6ª planta", "7ª planta", 
                                       "8ª planta", "9ª planta", "10ª planta",  "11ª planta", "12ª planta",
                                       "13ª planta", "14ª planta", "A partir de la 15ª planta"])
    elif c == "orientation":
        df[c] = pd.Categorical(df[c], ["Null", "Este", "Noreste", "Norte", "Noroeste", "Oeste", "Suroeste", "Sur", "Sureste"])
    else:
        df[c] = pd.Categorical(df[c])

group_cols = [e for e in cat_cols if e not in ("active", "formatted_address")]

In [ ]:
# target: price
# 
# correlations and feature importance (Chi-Square Test (For Ordinal Categorical Variables), Correlation Ratio (η): or Kruskal-Wallis Test for rest, normal corr for numerical)
# try imputation (dummy indicator for categorical, MICE, Mean/Median, KNN for numerical)
# preprocessing, scaling, encoding, model, etc, in pipeline
# try pipeline with different imputations/moodels